In [38]:
# Inspired by https://github.com/srush/Tensor-Puzzles
from tinygrad import Tensor
import numpy as np

In [ ]:
# These puzzles are about broadcasting. Know this rule.
# Each puzzle needs to be solved in 1 line (<80 columns) of code.
# You are allowed @, arithmetic, comparison, shape, any indexing (e.g. a[:j], a[:, None], a[arange(10)]), and previous puzzle functions.
# You are not allowed anything else. No view, sum, take, squeeze, tensor.

In [6]:
# arange
print(Tensor.arange(3))

<Tensor <LB METAL (3,) int (<BinaryOps.ADD: 1>, None)> on METAL with grad None>


In [8]:
# Example of broadcasting.
examples = [(Tensor.arange(4), Tensor.arange(5)[:, None]) ,
            (Tensor.arange(3)[:, None], Tensor.arange(2))]

print([a + b for a,b in examples])

[<Tensor <LB METAL (5, 4) int (<BinaryOps.ADD: 1>, None)> on METAL with grad None>, <Tensor <LB METAL (3, 2) int (<BinaryOps.ADD: 1>, None)> on METAL with grad None>]


In [12]:
# where
examples = [(Tensor([False]), Tensor([10]), Tensor([0])),
            (Tensor([False, True]), Tensor([1, 1]), Tensor([-10, 0])),
            (Tensor([False, True]), Tensor([1]), Tensor([-10, 0])),
            (Tensor([[False, True], [True, False]]), Tensor([1]), Tensor([-10, 0])),
            (Tensor([[False, True], [True, False]]), Tensor([[0], [10]]), Tensor([-10, 0])),
           ]

[q.where(a, b).numpy() for q, a, b in examples]

[array([0], dtype=int32),
 array([-10,   1], dtype=int32),
 array([-10,   1], dtype=int32),
 array([[-10,   1],
        [  1,   0]], dtype=int32),
 array([[-10,   0],
        [ 10,   0]], dtype=int32)]

In [40]:
# Puzzle 1 - ones
def ones(i) -> Tensor:
  # return Tensor.arange(i) - Tensor.arange(i) + 1
  return Tensor.where(Tensor.arange(i) > -1, 1, 0)

assert np.array_equal(ones(5).numpy(), np.ones(5))

In [37]:
# Puzzle 2 - sum
def sum(a: Tensor) -> Tensor:
  return a @ ones(a.shape[0])

a = Tensor([1, 2, 4, 5])
assert sum(a).numpy() == a.sum().numpy()

In [50]:
# Puzzle 3 - outer
def outer(a: Tensor, b: Tensor) -> Tensor:
  return a[:, None] * b[None, :]

a = Tensor.randint(6)
b = Tensor.randint(3)
assert np.array_equal(outer(a, b).numpy(), np.outer(a.numpy(), b.numpy()))
